# PiSynth 10.3

### Markus Kankkonen

## Getting started with Supriya
- Create Server, Boot server and create a Group

In [5]:
import supriya

server = supriya.Server.default()
server.boot()

group = supriya.realtime.Group().allocate()


###  Creating a synthdef

- Create a builder object with arguments
- with builder: creates the logic of the synth
- synthdef = builder.build().allocate() sends the synthdef to the server.
- server.sync() syncs the server

In [6]:
builder = supriya.synthdefs.SynthDefBuilder( amplitude=1.0, frequency=440.0, gate=1.0, )

with builder: 
  source = supriya.ugens.SinOsc.ar( frequency = builder['frequency'],) 
  envelope = supriya.ugens.EnvGen.kr( done_action = supriya.DoneAction.FREE_SYNTH,envelope = supriya.synthdefs.Envelope.asr(), gate = builder['gate'], ) 
  source = source * builder['amplitude']
  source = source * envelope 
  out = supriya.ugens.Out.ar( bus=0, source=source, )

synthdef = builder.build().allocate()

server.sync()


<Server: udp://127.0.0.1:57110, 8i8o>

### Creating a synth out of the synthDef

In [7]:

synth = supriya.Synth(synthdef)
synth

<- Synth: ???>

### Allocate the synth to the server on the previously created group

In [8]:
group.append(synth)
synth

<+ Synth: 1001>

### Querying the servers node tree

In [9]:
response = server.query_remote_nodes(include_controls=True)
print(response)

NODE TREE 0 group
    1 group
        1000 group
            1001 7095304975372a32297e8dd4dc6eff11
                amplitude: 1.0, frequency: 440.0, gate: 1.0


### Release synth and quit server

In [10]:
synth.release()

<+ Synth: 1001>

In [ ]:
server.quit()

- I can't get this to work on Windows, problem with some C function that is not available on non-unix platforms.
- On Linux to get some sound out of the server you need to setup your Jack connection so that the out from the Supercollider server goes to the systems playback. When you start a new one Jack will most likely default to not being connected. 

<img src="jack_connections.png">

- Get group with controls for each node

```
    response = s.query_remote_nodes(include_controls=True)
    print(response)
```
- Get id of a synth & how to send message to that synth.
- You have to sync the server otherwise it won't do anything
```
    synth_id = synth.node_id
    s.send(["/n_set",synth_id,"frequency",1000])
    s.sync()
```
- LinLin
```
    supriya.ugens.LinLin.ar([0,1,2,3,4,5],0,5,0,10)
```